# Notebook Initialization

In [1]:
import sys.process._
import ammonite.ops._
import ammonite.ops.ImplicitWd
// We need to create a temporary folder. We add a random in case you are working with several notebooks
val random = scala.util.Random.nextInt(10000)
val replClassPath = System.getProperty("user.home") + s"/replClasses$random"

s"mkdir -p $replClassPath" !


import sys.process._

import ammonite.ops._

import ammonite.ops.ImplicitWd
// We need to create a temporary folder. We add a random in case you are working with several notebooks

random: Int = 9711
replClassPath: String = "/home/jovyan/replClasses9711"
res0_5: Int = 0

In [2]:
interp.configureCompiler(_.settings.outputDirs.setSingleOutput(replClassPath))
interp.configureCompiler(_.settings.Yreplclassbased)
interp.load.cp(os.Path(replClassPath))

In [3]:
// Import the Snowpark library from Maven.
import $ivy.`com.snowflake:snowpark:0.10.0`

import com.snowflake.snowpark._
import com.snowflake.snowpark.functions._



import $ivy.$                              


import com.snowflake.snowpark._

import com.snowflake.snowpark.functions._



In [ ]:
## Creating a session

In [4]:
val session = Session.builder.configs(Map(
    "URL" -> ("https://"+System.getenv("SNOW_ACCOUNT") + ".snowflakecomputing.com"),
    "USER" -> System.getenv("SNOW_USER"),
    "PASSWORD" -> System.getenv("SNOW_PASSWORD"),
    "ROLE" -> System.getenv("SNOW_ROLE"),
    "WAREHOUSE" -> System.getenv("SNOW_WAREHOUSE"),
    "DB" -> System.getenv("SNOW_DATABASE")
)).create

// Add the directory for REPL classes that you created earlier.
session.addDependency(replClassPath)
var schema = System.getenv("SNOW_SCHEMA")
if (schema == null)
    schema = "PUBLIC"
session.sql(s"USE SCHEMA $schema").collect() 

[scala-interpreter-1] INFO com.snowflake.snowpark.Session - Closing stderr and redirecting to stdout


[scala-interpreter-1] INFO com.snowflake.snowpark.Session - Done closing stderr and redirecting to stdout
[scala-interpreter-1] INFO com.snowflake.snowpark.internal.ParameterUtils - set JDBC client memory limit to 10240
[scala-interpreter-1] INFO com.snowflake.snowpark.Session - Snowpark Session information: {
 "snowpark.version" : "0.10.0",
 "java.version" : "11.0.11",
 "scala.version" : "2.12.11",
 "jdbc.session.id" : "31151406055434",
 "os.name" : "Linux",
 "jdbc.version" : "3.13.7",
 "snowpark.library" : "/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/snowflake/snowpark/0.10.0/snowpark-0.10.0.jar",
 "scala.library" : "/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.11/scala-library-2.12.11.jar",
 "jdbc.library" : "/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.13.7/snowflake-jdbc-3.13.7.jar"
}
[scala-interpreter-1] INFO com.snowflake.snowpark.internal.ServerConnection - Act

session: Session = com.snowflake.snowpark.Session@11bcab9b
schema: String = "PUBLIC"
res3_4: Array[Row] = Array(Row[Statement executed successfully.])

## UDF Dependencies

In [5]:
// Adding the Ammonite kernel classes as dependencies for your UDF
def addClass(session: Session, className: String): String = {
  var cls1 = Class.forName(className)
  val resourceName = "/" + cls1.getName().replace(".", "/") + ".class"
  val url = cls1.getResource(resourceName)
  val path = url.getPath().split(":").last.split("!").head
  session.addDependency(path)
  path
}
addClass(session, "ammonite.repl.ReplBridge$")
addClass(session, "ammonite.interp.api.APIHolder")
addClass(session, "pprint.TPrintColors")

defined function addClass
res4_1: String = "/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/ammonite-repl-api_2.12.11/2.2.0-4-4bd225e/ammonite-repl-api_2.12.11-2.2.0-4-4bd225e.jar"
res4_2: String = "/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/ammonite-interp-api_2.12.11/2.2.0-4-4bd225e/ammonite-interp-api_2.12.11-2.2.0-4-4bd225e.jar"
res4_3: String = "/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/pprint_2.12/0.5.9/pprint_2.12-0.5.9.jar"

## UDF Validation

In [6]:
// Now let's try a simple UDF

class UDFCode extends Serializable {
  val appendLastNameFunc = (s: String) => {
    s"$s Johnson"
  }
}
// Define an anonymous UDF.
val appendLastNameUdf = udf((new UDFCode).appendLastNameFunc)
// Create a DataFrame that has a column NAME with a single row with the value "Raymond".
val df = session.sql("select 'Raymond' NAME")
// Call the UDF, passing in the values in the NAME column.
// Return a new DataFrame that has an additional column "Full Name" that contains the value returned by the UDF.
df.withColumn("Full Name", appendLastNameUdf(col("NAME"))).show()


-----------------------------
|"NAME"   |"Full Name"      |
-----------------------------
|Raymond  |Raymond Johnson  |
-----------------------------



defined class UDFCode
appendLastNameUdf: UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd5$Helper$UDFCode$$Lambda$3106/0x0000000100f77040@3da388b5,
  UdfColumnSchema(String, false),
  List(UdfColumnSchema(String, false)),
  Some("\"DEMODB\".\"PUBLIC\".SNOWPARK_TEMP_FUNCTION_RFERAXYWQD8BJLL")
)
df: DataFrame = com.snowflake.snowpark.DataFrame@449f0c71

# Preparing some sample Data

In [7]:
session.sql("create or replace table sample_product_data (id int, parent_id int, category_id int, name varchar, serial_number varchar, key int, \"3rd\" int);").collect()
session.sql("""insert into sample_product_data values
    (1, 0, 5, 'Product 1', 'prod-1', 1, 10),
    (2, 1, 5, 'Product 1A', 'prod-1-A', 1, 20),
    (3, 1, 5, 'Product 1B', 'prod-1-B', 1, 30),
    (4, 0, 10, 'Product 2', 'prod-2', 2, 40),
    (5, 4, 10, 'Product 2A', 'prod-2-A', 2, 50),
    (6, 4, 10, 'Product 2B', 'prod-2-B', 2, 60),
    (7, 0, 20, 'Product 3', 'prod-3', 3, 70),
    (8, 7, 20, 'Product 3A', 'prod-3-A', 3, 80),
    (9, 7, 20, 'Product 3B', 'prod-3-B', 3, 90),
    (10, 0, 50, 'Product 4', 'prod-4', 4, 100),
    (11, 10, 50, 'Product 4A', 'prod-4-A', 4, 100),
    (12, 10, 50, 'Product 4B', 'prod-4-B', 4, 100);""").collect()


res6_0: Array[Row] = Array(Row[Table SAMPLE_PRODUCT_DATA successfully created.])
res6_1: Array[Row] = Array(Row[12])

# Constructing a DataFrame

In [8]:
//To create a DataFrame from data in a table, view, or stream, call the table method:

// Create a DataFrame from the data in the "sample_product_data" table.
val dfTable = session.table("sample_product_data")

// To print out the first 10 rows, call:
dfTable.show()

-------------------------------------------------------------------------------------
|"ID"  |"PARENT_ID"  |"CATEGORY_ID"  |"NAME"      |"SERIAL_NUMBER"  |"KEY"  |"3rd"  |
-------------------------------------------------------------------------------------
|1     |0            |5              |Product 1   |prod-1           |1      |10     |
|2     |1            |5              |Product 1A  |prod-1-A         |1      |20     |
|3     |1            |5              |Product 1B  |prod-1-B         |1      |30     |
|4     |0            |10             |Product 2   |prod-2           |2      |40     |
|5     |4            |10             |Product 2A  |prod-2-A         |2      |50     |
|6     |4            |10             |Product 2B  |prod-2-B         |2      |60     |
|7     |0            |20             |Product 3   |prod-3           |3      |70     |
|8     |7            |20             |Product 3A  |prod-3-A         |3      |80     |
|9     |7            |20             |Product 3B  |pro

dfTable: Updatable = com.snowflake.snowpark.Updatable@75ea7b01

In [9]:
// Create a DataFrame containing a sequence of values.
// In the DataFrame, name the columns "i" and "s".
val dfSeq = session.createDataFrame(Seq((1, "one"), (2, "two"))).toDF("i", "s")

dfSeq: DataFrame = com.snowflake.snowpark.DataFrame@4b751158

In [ ]:
// Create a DataFrame from a range
val dfRange = session.range(1, 10, 2)

In [10]:
// Create a DataFrame from a SQL query
val dfSql = session.sql("SELECT name from products")

dfSql: DataFrame = com.snowflake.snowpark.DataFrame@3a77ae74

In [11]:
// To specify which rows should be returned, call the filter method:

// Import the col function from the functions object.
import com.snowflake.snowpark.functions._

// Create a DataFrame for the rows with the ID 1
// in the "sample_product_data" table.
//
// This example uses the === operator of the Column object to perform an
// equality check.
val df = session.table("sample_product_data").filter(col("id") === 1)
df.show()

------------------------------------------------------------------------------------
|"ID"  |"PARENT_ID"  |"CATEGORY_ID"  |"NAME"     |"SERIAL_NUMBER"  |"KEY"  |"3rd"  |
------------------------------------------------------------------------------------
|1     |0            |5              |Product 1  |prod-1           |1      |10     |
------------------------------------------------------------------------------------



import com.snowflake.snowpark.functions._

// Create a DataFrame for the rows with the ID 1
// in the "sample_product_data" table.
//
// This example uses the === operator of the Column object to perform an
// equality check.

df: DataFrame = com.snowflake.snowpark.DataFrame@6bc1e8ac

In [12]:
// Import the col function from the functions object.
import com.snowflake.snowpark.functions._

// Create a DataFrame that contains the id, name, and serial_number
// columns in te "sample_product_data" table.
val df = session.table("sample_product_data").select(col("id"), col("name"), col("serial_number"))
df.show()

---------------------------------------
|"ID"  |"NAME"      |"SERIAL_NUMBER"  |
---------------------------------------
|1     |Product 1   |prod-1           |
|2     |Product 1A  |prod-1-A         |
|3     |Product 1B  |prod-1-B         |
|4     |Product 2   |prod-2           |
|5     |Product 2A  |prod-2-A         |
|6     |Product 2B  |prod-2-B         |
|7     |Product 3   |prod-3           |
|8     |Product 3A  |prod-3-A         |
|9     |Product 3B  |prod-3-B         |
|10    |Product 4   |prod-4           |
---------------------------------------



import com.snowflake.snowpark.functions._

// Create a DataFrame that contains the id, name, and serial_number
// columns in te "sample_product_data" table.

df: DataFrame = com.snowflake.snowpark.DataFrame@5274821

In [13]:
// Create a DataFrame object for the "sample_product_data" table for the left-hand side of the join.
val dfLhs = session.table("sample_product_data")
// Clone the DataFrame object to use as the right-hand side of the join.
val dfRhs = dfLhs.clone()

// Create a DataFrame that joins the two DataFrames
// for the "sample_product_data" table on the
// "id" and "parent_id" columns.
val dfJoined = dfLhs.join(dfRhs, dfLhs.col("id") === dfRhs.col("parent_id"))
dfJoined.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"l_veuZ_ID"  |"l_veuZ_PARENT_ID"  |"l_veuZ_CATEGORY_ID"  |"l_veuZ_NAME"  |"l_veuZ_SERIAL_NUMBER"  |"l_veuZ_KEY"  |"l_veuZ_3rd"  |"r_UxpM_ID"  |"r_UxpM_PARENT_ID"  |"r_UxpM_CATEGORY_ID"  |"r_UxpM_NAME"  |"r_UxpM_SERIAL_NUMBER"  |"r_UxpM_KEY"  |"r_UxpM_3rd"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1            |0                   |5                     |Product 1      |prod-1                  |1             |10            |2            |1                   |5                     |Product 1A     |prod-1-A        

dfLhs: Updatable = com.snowflake.snowpark.Updatable@4f2a1653
dfRhs: Updatable = com.snowflake.snowpark.Updatable@5329792
dfJoined: DataFrame = com.snowflake.snowpark.DataFrame@3a9618a9

In [14]:
// Specifying Columns and Expressions
// When calling these transformation methods, you might need to specify columns or expressions that use columns. For example, when calling the select method, you need to specify the columns that should be selected.

// To refer to a column, create a Column object by calling the col function in the com.snowflake.snowpark.functions object.

// Import the col function from the functions object.
import com.snowflake.snowpark.functions._

val dfProductInfo = session.table("sample_product_data").select(col("id"), col("name"))
dfProductInfo.show()

---------------------
|"ID"  |"NAME"      |
---------------------
|1     |Product 1   |
|2     |Product 1A  |
|3     |Product 1B  |
|4     |Product 2   |
|5     |Product 2A  |
|6     |Product 2B  |
|7     |Product 3   |
|8     |Product 3A  |
|9     |Product 3B  |
|10    |Product 4   |
---------------------



import com.snowflake.snowpark.functions._


dfProductInfo: DataFrame = com.snowflake.snowpark.DataFrame@3c513c0c

In [15]:
// When specifying a filter, projection, join condition, etc., you can use Column objects in an expression. For example:
val df = session.table("sample_product_data")
// You can use Column objects with the filter method to specify a filter condition:

// Specify the equivalent of "WHERE id = 20"
// in an SQL SELECT statement.
df.filter(col("id") === 20)
// Specify the equivalent of "WHERE a + b < 10"
// in an SQL SELECT statement.
df.filter((col("a") + col("b")) < 10)
//You can use Column objects with the select method to define an alias:

// Specify the equivalent of "SELECT b * 10 AS c"
// in an SQL SELECT statement.
df.select((col("b") * 10) as "c")

df: Updatable = com.snowflake.snowpark.Updatable@1bdefe41
res14_1: DataFrame = com.snowflake.snowpark.DataFrame@4285fab3
res14_2: DataFrame = com.snowflake.snowpark.DataFrame@2216b3a5
res14_3: DataFrame = com.snowflake.snowpark.DataFrame@21c402df

In [16]:
//Retrieving Column Definitions
//To retrieve the definition of the columns in the dataset for the DataFrame, call the schema method. This method returns a StructType object that contains an Array of StructField objects. Each StructField object contains the definition of a column.

// Get the StructType object that describes the columns in the
// underlying rowset.
val tableSchema = session.table("sample_product_data").schema
println("Schema for sample_product_data: " + tableSchema);

Schema for sample_product_data: StructType[StructField(ID, Long, Nullable = true), StructField(PARENT_ID, Long, Nullable = true), StructField(CATEGORY_ID, Long, Nullable = true), StructField(NAME, String, Nullable = true), StructField(SERIAL_NUMBER, String, Nullable = true), StructField(KEY, Long, Nullable = true), StructField("3rd", Long, Nullable = true)]


tableSchema: types.StructType = StructType(
  StructField(ID, Long, true),
  StructField(PARENT_ID, Long, true),
  StructField(CATEGORY_ID, Long, true),
  StructField(NAME, String, true),
  StructField(SERIAL_NUMBER, String, true),
  StructField(KEY, Long, true),
  StructField("3rd", Long, true)
)

In [17]:
//To execute the query and return the number of results, call the count method:

// Create a DataFrame for the "sample_product_data" table.
val dfProducts = session.table("sample_product_data")

// Send the query to the server for execution and
// print the count of rows in the table.
println("Rows returned: " + dfProducts.count())

Rows returned: 12


dfProducts: Updatable = com.snowflake.snowpark.Updatable@1ea168f8

Call update to update existing rows in the table. See [Updating Rows in a Table.](https://docs.snowflake.com/en/developer-guide/snowpark/working-with-dataframes.html#label-snowpark-dataframe-update-table-rows)

Call delete to delete rows from a table. See [Deleting Rows in a Table.](https://docs.snowflake.com/en/developer-guide/snowpark/working-with-dataframes.html#label-snowpark-dataframe-delete-table-rows)

Call merge to insert, update, and delete rows in one table, based on data in a second table or subquery. (This is the equivalent of the MERGE command in SQL.) See [Merging Rows into a Table.](https://docs.snowflake.com/en/developer-guide/snowpark/working-with-dataframes.html#label-snowpark-dataframe-merge-table-rows)

In [ ]:
//For example, to replace the values in the column named count with the value 1:

val updatableDf = session.table("sample_product_data")
val updateResult = updatableDf.update(Map("count" -> lit(1)))
println(s"Number of rows updated: ${updateResult.rowsUpdated}")